**Mount drive**

Source: Mounting Google Drive: https://colab.research.google.com/notebooks/io.ipynb

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Imports**

In [0]:
import tensorflow as tf
import keras
import os
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.utils import plot_model
from keras.callbacks import EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

**Adding layers on top of VGG16**

Sources:

Building network: https://keras.io/applications/

Layers: https://riptutorial.com/keras/example/32608/transfer-learning-using-keras-and-vgg

Plotting the architecture: https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/

In [0]:
# build model

batch_size_train_and_validation = 32
number_of_epochs = 50

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#print(base_model.summary())
extra_layers = base_model.output

#extra_layers = GlobalAveragePooling2D()(extra_layers)
#extra_layers = Dropout(0.5)(extra_layers)
#extra_layers = Dense(1024, activation='relu')(extra_layers)
#extra_layers = Dropout(0.5)(extra_layers)
#final_layer = Dense(34, activation='softmax')(extra_layers)

#extra_layers = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(extra_layers)
#extra_layers = MaxPooling2D(pool_size=(2, 2))(extra_layers)
extra_layers = BatchNormalization()(extra_layers)

extra_layers = Flatten()(extra_layers)
extra_layers = Dropout(0.5)(extra_layers) #added layer
extra_layers = Dense(512, activation='relu')(extra_layers)
extra_layers = Dropout(0.5)(extra_layers)
extra_layers = Dense(256, activation='relu')(extra_layers)
extra_layers = Dropout(0.5)(extra_layers)
final_layer = Dense(34, activation='softmax')(extra_layers)
# 34 denotes the number of classifications we want to make

model = Model(inputs=base_model.input, outputs=final_layer)

#print('New model')
#print(model.summary())

# plot model
plot_model(model, to_file='VGG_BatchNormalisationBeforeAddedLayers.png', show_shapes=True, show_layer_names=True)

**Checking layers**

In [0]:
for layer in model.layers:
  print(layer.name)

In [0]:
for layer in base_model.layers:
    print(layer.name)
    
print(model.summary())

print("layers to freeze:")
for layer in base_model.layers[:15]:
  #layer.trainable = False
  print(layer.name)

print("layers to train:")  
for layer in base_model.layers[15:]:
  print(layer.name)

**Compile and Train/Fine-tune new model**

Sources:

Freezing layers and compiling: https://keras.io/applications/

Image Data Generator: https://keras.io/preprocessing/image/ , https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

Validation split and subsets: https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator/52372042#52372042

Getting the class indeces: https://stackoverflow.com/questions/38971293/get-class-labels-from-keras-functional-model

Generators, flow_from_directory, training the model: https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

EarlyStopping: https://keras.io/callbacks/#earlystopping

In [0]:
# freeze layers
for layer in base_model.layers[:15]:
  layer.trainable = False
for layer in base_model.layers[15:]:
  layer.trainable = True

image_target_size = (224, 224)  
  
# compile model  
#model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])
model.compile(SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])
#model.compile(RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

train_datagenerator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                         rescale=1./255,
                                         zoom_range=0.1,
                                         shear_range=0.1,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         rotation_range=20,
                                         validation_split=0.2)  #validation split 20/80

directory_path = 'drive/My Drive/Monkey Faces/New Split/train'

train_generator=train_datagenerator.flow_from_directory(directory_path,
                                                        color_mode = 'rgb',
                                                        class_mode ='categorical',
                                                        batch_size = batch_size_train_and_validation,
                                                        shuffle = True,
                                                        subset = 'training',
                                                        target_size=image_target_size)
validation_generator=train_datagenerator.flow_from_directory(directory_path,
                                                             color_mode = 'rgb',
                                                             class_mode = 'categorical',
                                                             batch_size = batch_size_train_and_validation,
                                                             shuffle = True,
                                                             subset = 'validation',
                                                             target_size=image_target_size)

#get the class for each monkey as an index from the flow_from_directory
label_map = (train_generator.class_indices) 
#print(label_map)
#print(directory_path)

# train model
step_size_train = train_generator.n//train_generator.batch_size 
# steps per epoch is total number of pictures divided by the batch size
step_size_validation = validation_generator.n//train_generator.batch_size 
# steps for validation is calculated same way

# earlystopping in case accuracy does not improve during training
early_stopping = EarlyStopping(monitor='val_acc',
                               min_delta=0.01,
                               patience=20,
                               mode='auto',
                               restore_best_weights=True)

model_gen = model.fit_generator(generator = train_generator,
                    steps_per_epoch = step_size_train,
                    validation_data = validation_generator,
                    validation_steps = step_size_validation, 
                    epochs = number_of_epochs,
                    shuffle = True,
                    callbacks=[early_stopping]) 
   
model.save('VGG16, Fully Trained, Batch Normalisation Before Flatten Layer.h5')

**Evaluate model before fine-tuning layers from VGG16**

Sources:

Test data generator, test steps, validation steps: https://stackoverflow.com/questions/45806669/keras-how-to-use-predict-generator-with-imagedatagenerator

In [0]:
image_target_size = (224, 224)

test_directory_path = 'drive/My Drive/Monkey Faces/New Split/test (With subfolders)'

test_datagenerator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rescale=1./255)

test_generator = test_datagenerator.flow_from_directory(test_directory_path,
                                                        color_mode = 'rgb',
                                                        class_mode ='categorical',
                                                        batch_size = 1,
                                                        shuffle = False,
                                                        target_size=image_target_size)

test_steps = len(test_generator.filenames)
validation_steps = len(validation_generator.filenames)

# evaluate validation generator
print("Validation evaluation:")
validation_evaluation = model.evaluate_generator(validation_generator, validation_steps)
print("Loss: ", validation_evaluation[0], ", ", "Top-1 Accuracy: ", validation_evaluation[1], ", ",
     "Top-5 Accuracy: ", validation_evaluation[2])

# evaluate test generator
print("Test evaluation:")
test_evaluation = model.evaluate_generator(test_generator, steps = test_steps)
print("Loss: ", test_evaluation[0], ", ", "Top-1 Accuracy: ", test_evaluation[1], ", ",
     "Top-5 Accuracy: ", test_evaluation[2])

**Make predictions after fine-tuning model**

Source:

predict_generator: https://stackoverflow.com/questions/45806669/keras-how-to-use-predict-generator-with-imagedatagenerator

In [0]:
test_generator.reset()

predictions = model.predict_generator(test_generator, steps = test_steps)

# index for iterating through file names produced by generator, to print out the monkey
# because shuffle=False, the train_generator will take the files using next() and therefore
# in the order printed by the monkey_names
monkey_index = 0

for individual_prediction in predictions:
  print(test_generator.filenames[monkey_index].partition('/')[0])
  #print(individual_prediction)
  predictions_descending = (-individual_prediction).argsort()
  #print(predictions_descending)
  top_5_predictions = predictions_descending[:5]
  #print(top_5_predictions)

  top_5_monkeys = {}
  for monkey_name, id_in_map in label_map.items():
    # find the monkeys from label_map which are in top 5
    if (id_in_map in top_5_predictions): 
      # cast top 5 monkeys to a dictionary
      top_5_monkeys[monkey_name, id_in_map] = individual_prediction[id_in_map]
      
  for monkey in sorted(top_5_monkeys, key=top_5_monkeys.get, reverse=True):
    print(monkey, top_5_monkeys[monkey])

  monkey_index += 1 

**Plot graphs**

Source: Plotting graph: https://keras.io/visualization/

In [0]:
import matplotlib.pyplot as plt

plt.plot(model_gen.history['acc'])
plt.plot(model_gen.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(model_gen.history['loss'])
plt.plot(model_gen.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

**Plot confusion matrix**

Sources:

Confusion matrix: https://groups.google.com/forum/#!topic/keras-users/bqWwFox_zZs , https://www.kaggle.com/amarjeet007/visualize-cnn-with-keras

In [0]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# plot confusion matrix
classes = test_generator.classes[test_generator.index_array]
top_prediction = np.argmax(predictions, axis=-1)
conf_matrix = confusion_matrix(test_generator.classes[test_generator.index_array], top_prediction)

plt.figure(figsize=(15,15))
sns.heatmap(conf_matrix, annot=True, fmt="d")